In [1]:
# train_utils.py

import torch
from torch import nn
from torch.autograd import Variable
import copy
import math
import numpy as np

def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

class Embeddings(nn.Module):
    '''
    Usual Embedding layer with weights multiplied by sqrt(d_model)
    '''
    def __init__(self,embeddings ,d_model, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        #self.lut = nn.Embedding.from_pretrained(embeddings, freeze=True)
        
        self.d_model = d_model

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)

class PositionalEncoding(nn.Module):
    "Implement the PE function."
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() *
                             -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(torch.as_tensor(position.numpy() * div_term.unsqueeze(0).numpy()))
        pe[:, 1::2] = torch.cos(torch.as_tensor(position.numpy() * div_term.unsqueeze(0).numpy()))#torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + Variable(self.pe[:, :x.size(1)],
                         requires_grad=False)
        return self.dropout(x)
# attention.py

import torch
from torch import nn
import math
import torch.nn.functional as F


def attention(query, key, value, mask=None, dropout=None):
    "Implementation of Scaled dot product attention"
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = F.softmax(scores, dim = -1)
    if dropout is not None:
        p_attn = dropout(p_attn)

    return torch.matmul(p_attn, value), p_attn

class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        "Take in model size and number of heads."
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), self.h)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, query, key, value, mask=None):
        "Implements Multi-head attention"
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)

        # 1) Do all the linear projections in batch from d_model => h x d_k
        query, key, value = \
            [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
             for l, x in zip(self.linears, (query, key, value))]

        # 2) Apply attention on all the projected vectors in batch.
        x, self.attn = attention(query, key, value, mask=mask,
                                 dropout=self.dropout)

        # 3) "Concat" using a view and apply a final linear.
        x = x.transpose(1, 2).contiguous() \
             .view(nbatches, -1, self.h * self.d_k)
        return self.linears[-1](x)
class Encoder(nn.Module):
    '''
    Transformer Encoder

    It is a stack of N layers.
    '''
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)

    def forward(self, x, mask=None):
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

class EncoderLayer(nn.Module):
    '''
    An encoder layer

    Made up of self-attention and a feed forward layer.
    Each of these sublayers have residual and layer norm, implemented by SublayerOutput.
    '''
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer_output = clones(SublayerOutput(size, dropout), 2)
        self.size = size

    def forward(self, x, mask=None):
        "Transformer Encoder"
        x = self.sublayer_output[0](x, lambda x: self.self_attn(x, x, x, mask)) # Encoder self-attention
        return self.sublayer_output[1](x, self.feed_forward)
# sublayer.py

import torch
from torch import nn

class LayerNorm(nn.Module):
    "Construct a layer normalization module."
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

class SublayerOutput(nn.Module):
    '''
    A residual connection followed by a layer norm.
    '''
    def __init__(self, size, dropout):
        super(SublayerOutput, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        "Apply residual connection to any sublayer with the same size."
        return x + self.dropout(sublayer(self.norm(x)))
# feed_forward.py

from torch import nn
import torch.nn.functional as F

class PositionwiseFeedForward(nn.Module):
    "Positionwise feed-forward network."
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        "Implements FFN equation."
        return self.w_2(self.dropout(F.relu(self.w_1(x))))
# Model.py

import torch
import torch.nn as nn
from copy import deepcopy


class Transformer(nn.Module):
    def __init__(self, config, src_vocab):
        super(Transformer, self).__init__()
        self.config = config

        h, N, dropout = self.config.h, self.config.N, self.config.dropout
        d_model, d_ff = self.config.d_model, self.config.d_ff

        attn = MultiHeadedAttention(h, d_model)
        ff = PositionwiseFeedForward(d_model, d_ff, dropout)
        position = PositionalEncoding(d_model, dropout)
        self.src_vocab = src_vocab
        self.encoder = Encoder(EncoderLayer(config.d_model, deepcopy(attn), deepcopy(ff), dropout), N)
        
        a = dataset.vocab.vectors.tolist()
        for i in range(len(a)):
            while(len(a[i])!=config.d_model):
                a[i].append(0)
                
        a= torch.tensor(a)
               
        self.src_embed = nn.Sequential(Embeddings(dataset.vocab,config.d_model, len(self.src_vocab)), deepcopy(position)) #Embeddings followed by PE

        # Fully-Connected Layer
        self.fc = nn.Linear(
            self.config.d_model,
            self.config.output_size
        )

        # Softmax non-linearity
        self.softmax = nn.Softmax()
        self.relu = nn.ReLU()
    def forward(self, x):    
        
        embedded_sents = self.src_embed(x.permute(1,0)) # shape = (batch_size, sen_len, d_model)
        
        encoded_sents = self.encoder(embedded_sents)

        # Convert input to (batch_size, d_model) for linear layer
        
        
        final_feature_map = encoded_sents[:,-1,:]


        final_out = self.fc(final_feature_map)
        final_out = self.softmax(final_out)
        
        return final_out

    def add_optimizer(self, optimizer):
        self.optimizer = optimizer

    def add_loss_op(self, loss_op):
        self.loss_op = loss_op

    def reduce_lr(self):
        print("Reducing LR")
        for g in self.optimizer.param_groups:
            g['lr'] = g['lr'] / 2
    def predict(self,text):
        y_pred = self.__call__(text)
        
        return y_pred
    
    def run_epoch(self, train_iterator, val_iterator, epoch):
        train_losses = []
        val_accuracies = []
        losses = []

        # Reduce learning rate as number of epochs increase


        for i, batch in enumerate(train_iterator):


            if torch.cuda.is_available():
                x = batch.text.cuda()
                y = (batch.label-1).type(torch.LongTensor)
                
            else:
                x = batch.text
                y = (batch.label-1).type(torch.LongTensor)
            
            y_pred = self.__call__(x)

            loss = self.loss_op(y_pred, y)
            self.optimizer.zero_grad()
            loss.backward()
            losses.append(loss.data.cpu().numpy())
            self.optimizer.step()

            if i % 100 == 0:
                avg_train_loss = np.mean(losses)
                train_losses.append(avg_train_loss)
                print("\tAverage training loss: {:.5f}".format(avg_train_loss))
                losses = []

                # Evalute Accuracy on validation set
                val_accuracy,f1 = evaluate_model(self, val_iterator)
                print("\tVal Accuracy: {:.4f}".format(val_accuracy))
                print("\tf1: {:.4f}".format(f1))
                
                self.train()

        return train_losses, val_accuracies
        
    def run_epoch2(self, train_iterator,val_iterator,epoch):
        losses = []
        for j in range(epoch):
            for i, batch in enumerate(train_iterator):
                if torch.cuda.is_available():
                    x = batch.text.cuda()
                    y = (batch.label-1).type(torch.LongTensor)                
                else:
                    x = batch.text
                    y = (batch.label-1).type(torch.LongTensor)

                y_pred = self.__call__(x)
                print([torch.argmax(y_pred, dim=1),y])
                loss = self.loss_op(y_pred, y)
                self.optimizer.zero_grad()
                
                loss.backward()
                
                self.optimizer.step()
            losses.append(loss.detach().numpy())  
            
            if (j+1) % 1 == 0:
                print (f'Epoch [{j+1}/{epoch}], Loss: {loss.item():.10f}')
                # Evalute Accuracy on validation set
                val_accuracy,f1 = evaluate_model(self, val_iterator)
                print("\tVal Accuracy: {:.4f}".format(val_accuracy))
                print("\tf1: {:.4f}".format(f1))
                
                self.train()
                

                
        return losses
# config.py

class Config(object):
    N = 6 #6 in Transformer Paper
    d_model = 512 #512 in Transformer Paper
    d_ff = 2048 #2048 in Transformer Paper
    h = 8
    dropout = 0.1
    output_size = 4
    lr = 0.0003
    max_epochs = 200
    batch_size = 1
    max_sen_len = 50        

In [2]:
import telebot
import time
from telebot import types
import json
import torch

token = '6705181314:AAH1F4h1C_rpM5pkcu3tXdeHkznDxIESz3o'
bot = telebot.TeleBot(token, parse_mode='None')
bot_name = 'ITMO BOT'

model= torch.load('/Users/mac/Desktop/test/model.pth')
def menu():
        markup = types.InlineKeyboardMarkup()
        markup.add(types.InlineKeyboardButton('help', callback_data='help'))   
        markup.add(types.InlineKeyboardButton('contact', callback_data='contact'))   
        markup.add(types.InlineKeyboardButton('main questions', callback_data='main questions'))  
        markup.add(types.InlineKeyboardButton('more questions', callback_data='more questions')) 
        markup.add(types.InlineKeyboardButton('application', callback_data='application'))    
        return markup
def main():       
    @bot.message_handler(commands=['start'])  # Ответ на команду /start
    def start(message):
        mess = f'hi, <b>{message.from_user.first_name}</b>!\nI am - <b>{bot_name}</b>'
        markup = menu()
        bot.send_message(message.chat.id, mess, reply_markup=markup, parse_mode='html')

def restart(message):
    mess = f'hi, <b>{message.from_user.first_name}</b>!\nI am - <b>{bot_name}</b>'
    markup = menu()
    bot.send_message(message.chat.id, mess, reply_markup=markup, parse_mode='html')


In [3]:
with open('/Users/mac/Desktop/SCIENTIFIC RESEARCH/main QA.json', 'r') as json_data:
    main_intents = json.load(json_data)
corpse = []
responses = []
for intent in main_intents['intents']:
    tag = intent['tag']
    response = intent['responses']
    print(tag+"\n")
    corpse.append(tag)# here we are appending the word with its tag
    responses.append(response)

requirement

scholarships

batchmates

language

research topics

internship



In [4]:
from nltk.tokenize import regexp_tokenize
@bot.callback_query_handler(func=lambda call: True)
def message_reply(call):
    if call.data == 'contact':
        mess = 'click on mail to get contact with staff'
        bot.send_message(call.message.chat.id,mess)
        mess = """
Program coordinator --  aakarabintseva@itmo.ru
International office -- international@itmo.ru
Student office -- aakiseleva@itmo.ru
Migration office -- aakhalilova@itmo.ru 
"""
        bot.send_message(call.message.chat.id,mess)
        time.sleep(5)
        restart(call.message)
    elif call.data == 'help': 
        mess = """
contact --  to find Email address of specific staff in ITMO
main questions -- to answer most frequent questions from candidates
more questions -- to answer other questions
application -- to redirect to page to fill application
"""
        bot.send_message(call.message.chat.id,mess)
        time.sleep(5)
        restart(call.message)
    elif call.data == 'main questions':
        mess = 'please choose interested item'
        markup = types.ReplyKeyboardMarkup(one_time_keyboard=True)
        for i in range(len(corpse)):            
            markup.add(corpse[i])  
        msg = bot.send_message(call.message.chat.id,mess, reply_markup=markup)
        bot.register_next_step_handler(msg, main_questions)
    elif call.data == 'more questions':
        msg = bot.send_message(call.message.chat.id, 'please write to questions')
        bot.register_next_step_handler(msg, more_questions)
    elif call.data == 'application':
        linked_user = 'https://signup.itmo.ru/master'
        markup = types.InlineKeyboardMarkup()
        markup.add(types.InlineKeyboardButton(text='redirect to ITMO',
                            url=linked_user))
        mess = 'click to redirect to application form'
        bot.send_message(call.message.chat.id,mess, reply_markup=markup)
        time.sleep(5)
        restart(call.message)
    


def main_questions(message):
    def tokenize(sentence):
        return regexp_tokenize(sentence, pattern="\w+")

    def score_words(x,y):
          """ returns the jaccard similarity between two lists """
          intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
          union_cardinality = len(set.union(*[set(x), set(y)]))
          return intersection_cardinality/float(union_cardinality)
    sentence = message.text
    if(any(sentence.lower()==item.lower() for item in ["quit","finish","over","bye","goodbye"])):
        print(f"{bot_name}: Goodbye , have a nice day")

    similarity = []
    for i in corpse:
        similarity.append(score_words(sentence,i))
    #print(similarity)
    
    if(max(similarity) > 0.5 and len(tokenize(sentence))==1 ):
        print(f"{bot_name}: "+responses[similarity.index(max(similarity))][0])
    
    mess = responses[similarity.index(max(similarity))][0]
    bot.send_message(message.chat.id, mess)

    time.sleep(5)
    restart(message)
    

In [5]:
import spacy
import random


def more_questions(message):
    def tokenize(sentence):
        return regexp_tokenize(sentence, pattern="\w+")
    sentence = message.text
    NLP = spacy.blank("en")
    tokenizer = lambda sent: [x.text for x in NLP.tokenizer(sent) if x.text != " "]
    token = tokenizer(sentence)
    sequenced=[]
    for i in range(len(token)):
        sequenced.append(model.src_vocab[token[i]])
    
    while(len(sequenced)!=model.config.max_sen_len):
        sequenced.append(1)
    
    array = np.array(sequenced)
    
    transposed_array = array.T
    sequenced = transposed_array.tolist()
    tensor = []
    tensor.append(sequenced)
    print('i am token')
    print(token)
    user_input = torch.tensor(tensor)
    print(user_input)
    user_input = user_input.permute(1,0)
    
    output = model.predict(user_input)
    print(output)
    prob = output.max() 
    print(prob)
    output = torch.argmax(output, dim=1)
    
    output = output + 1
    print(output)
    mess = ''
    with open('/Users/mac/Desktop/test/test.json', 'r') as json_data:
        intents = json.load(json_data)
    if prob > 0.1:
            for intent in intents['intents']:
                if output == intent["tag"]:
                    mess = random.choice(intent['responses'])
                    print(random.choice(intent['responses']))
                
    else:
        mess = "Sorry I am unable to Process Your Request"
        mess += "You may find the way forward in https://en.itmo.ru/en/viewjep/2/5/Big_Data_and_Machine_Learning.htm"
    
    bot.send_message(message.chat.id, mess)
    time.sleep(5)
    restart(message)
if __name__ == "__main__":
        main()
        bot.polling(none_stop=True)

i am token
['What', 'are', 'the', 'requirements', 'to', 'enter', 'the', 'program', '?']
tensor([[  0,   9,   3, 190,   5,  47,   3,  12,   2,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1]])
tensor([[9.9942e-01, 1.1392e-05, 1.7274e-05, 1.3223e-05, 3.8645e-07, 1.0744e-05,
         1.8579e-06, 6.0522e-07, 3.6370e-05, 5.5307e-06, 2.1374e-07, 1.1548e-04,
         1.4297e-04, 1.8410e-05, 1.1575e-05, 9.5544e-05, 4.4703e-05, 3.3011e-07,
         8.2599e-07, 3.8206e-05, 2.2118e-07, 8.7527e-06, 2.1505e-07, 6.6277e-07]],
       grad_fn=<SoftmaxBackward0>)
tensor(0.9994, grad_fn=<MaxBackward1>)
tensor([1])
To enter the program applicants need to: – send a portfolio to the international department with their relevant professional (and academic) CV, transcript of records for their Bachelor's degree, and a proof of th

/opt/homebrew/Cellar/jupyterlab/4.0.6_1/libexec/lib/python3.11/site-packages/torch/nn/modules/module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


i am token
['Are', 'all', 'courses', 'at', 'the', 'program', 'taught', 'in', 'English', '?']
tensor([[  0, 109,  43,  17,   3,  12,  99,  15,   0,   2,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1]])
tensor([[1.4977e-05, 4.4491e-04, 2.1037e-04, 9.9335e-01, 3.1562e-06, 1.8712e-03,
         5.9387e-06, 5.8804e-04, 2.6249e-04, 2.7798e-06, 1.1834e-05, 8.3511e-06,
         8.9419e-04, 1.2096e-04, 3.5883e-04, 1.3767e-04, 1.2635e-03, 8.4131e-05,
         3.1195e-05, 2.1365e-05, 3.2253e-05, 9.0611e-06, 2.1210e-04, 6.1196e-05]],
       grad_fn=<SoftmaxBackward0>)
tensor(0.9933, grad_fn=<MaxBackward1>)
tensor([4])
Yes, 100% of courses offered at the program are taught in English.
i am token
['Will', 'I', 'get', 'the', 'chance', 'to', 'do', 'an', 'internship', 'during', 'my', 'studies', '?']
tensor([[  0,   0,  23,   3